In [15]:
import pandas as pd
technical_indicators_binary = pd.read_csv('/content/technical_indicators_binary.csv', index_col=0)

In [16]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import TimeSeriesSplit
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score, classification_report

# Normalize the features
scaler = MinMaxScaler()
X = scaler.fit_transform(technical_indicators_binary.drop(columns=['Label']))
y = technical_indicators_binary['Label'].values

# Reshape the input data into a 3D array as required by LSTM
n_features = X.shape[1]
X = X.reshape(-1, 1, n_features)

# Define the LSTM model
model = Sequential()
model.add(LSTM(units=150, activation='relu', input_shape=(1, n_features)))
model.add(Dropout(0.4))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

tscv = TimeSeriesSplit(n_splits=5)
accuracy_scores = []
y_true_all = []
y_pred_all = []

for train_index, test_index in tscv.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Train the model
    model.fit(X_train, y_train, epochs=200, batch_size=128, verbose=0)

    # Evaluate the model
    y_pred = (model.predict(X_test) > 0.5).astype("int32")
    accuracy = accuracy_score(y_test, y_pred)
    accuracy_scores.append(accuracy)

    y_true_all.extend(y_test)
    y_pred_all.extend(y_pred.flatten())

# Calculate the average accuracy
average_accuracy = np.mean(accuracy_scores)
print("Average accuracy:", average_accuracy)

# Print the classification report
print("\nClassification report:")
print(classification_report(y_true_all, y_pred_all))


26/26 [==============================] - 0s 2ms/step
Average accuracy: 0.5575757575757575

Classification report:
              precision    recall  f1-score   support

           0       0.55      0.45      0.49      2002
           1       0.56      0.66      0.61      2123

    accuracy                           0.56      4125
   macro avg       0.56      0.55      0.55      4125
weighted avg       0.56      0.56      0.55      4125

